# Create a Bucket and Lock it Down to a Specific VPC

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
s3 = boto3.Session().client(service_name='s3', region_name=region)

In [ ]:
import time

timestamp = int(time.time())

bucket_secure_name = 'bucket-secure-bucket-policy-vpc-{}'.format(timestamp)
print(bucket_secure_name)

In [ ]:
!aws s3 mb s3://$bucket_secure_name

In [7]:
!aws s3 ls s3://$bucket_secure_name

In [16]:
# Create the bucket policy
bucket_policy_deny = {
        "Version": "2008-10-17",
        "Statement": [
            {
                "Effect": "Deny",
                "Principal": "*",
                "Action": [
                    "s3:ListBucket"
                ],
                "Resource": [
                    "arn:aws:s3:::{}".format(bucket_secure_name)
                ],
                "Condition": {
                    "StringNotEquals": {
                        "aws:sourceVpc": vpc_id
                    }
                }
            }
        ]
    }

# Set the Deny Policy on the Bucket

In [19]:
import json
import time

response = s3.put_bucket_policy(Bucket=bucket_secure_name, 
                                Policy=json.dumps(bucket_policy_deny))

print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': 'DCC486FBC88E9551', 'HostId': '+UYdxcKmxMYXzRT83xNF+VlQcwS9hKNx0auqd6DvDFXdN2ocn0bcSXpOS76v+EwINjlyHovGy10=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': '+UYdxcKmxMYXzRT83xNF+VlQcwS9hKNx0auqd6DvDFXdN2ocn0bcSXpOS76v+EwINjlyHovGy10=', 'x-amz-request-id': 'DCC486FBC88E9551', 'date': 'Sun, 20 Dec 2020 22:51:38 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


# Verify `AccessDenied`

In [14]:
!aws s3 ls s3://$bucket_secure_name


An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


# Remove the Policy

In [21]:
response = s3.delete_bucket_policy(Bucket=bucket_secure_name)
print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': 'B73FAA6B270A28B6', 'HostId': 'eVRQSkytCL0osV89EIiVpjN6HF+w8dyQgUjcP06ocXJJUUN2peD66bA22Z+6cmj/QbCRfZ+5v5Q=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'eVRQSkytCL0osV89EIiVpjN6HF+w8dyQgUjcP06ocXJJUUN2peD66bA22Z+6cmj/QbCRfZ+5v5Q=', 'x-amz-request-id': 'B73FAA6B270A28B6', 'date': 'Sun, 20 Dec 2020 22:53:23 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


# Verify Access After Removing the Policy

In [22]:
!aws s3 ls s3://$bucket_secure_name

# Remove the Bucket

In [23]:
!aws s3 rm s3://$bucket_secure_name